# Von Digitalisaten zu Wissensgraphen: Eine automatisierte Extraktion und semantische Modellierung biographischer Daten am Beispiel von Lebensbeschreibungen der Herrnhuter Brüdergemeine

## OCR-Erkennung

- Daten werden als jpg importiert

In [21]:
import cv2
import os
import pytesseract

In [22]:
# Ordner mit den jpg-Dateien 
base_folder = "data/jpg"

# Ordner für die TXT-Dateien
output_base = "data/txt"

In [5]:
# Durchlaufe alle Unterordner
for folder_name in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder_name)

    # Nur Verzeichnisse berücksichtigen
    if not os.path.isdir(folder_path):
        continue

    # Alle .jpg-Dateien holen
    jpeg_files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith('.jpg')])

    if not jpeg_files:
        print(f"Keine .jpg-Dateien in Ordner: {folder_name}")
        continue

    # Zielunterordner erstellen, falls er nicht existiert
    output_folder = os.path.join(output_base, folder_name)
    os.makedirs(output_folder, exist_ok=True)

    # Bilder verarbeiten
    for jpeg_file in jpeg_files:
        img_path = os.path.join(folder_path, jpeg_file)
        img = cv2.imread(img_path)

        # Bildvorverarbeitung
        inverted_image = cv2.bitwise_not(img)
        gray_image = cv2.cvtColor(inverted_image, cv2.COLOR_BGR2GRAY)
        _, binary_image = cv2.threshold(gray_image, 120, 255, cv2.THRESH_BINARY)
        binary_image_contrast = cv2.convertScaleAbs(binary_image, alpha=2.0, beta=0)

        # OCR
        ocr_result = pytesseract.image_to_string(binary_image_contrast, lang="deu+frk")

        # TXT-Dateiname & Pfad
        txt_filename = os.path.splitext(jpeg_file)[0] + ".txt"
        txt_path = os.path.join(output_folder, txt_filename)

        # Speichern
        with open(txt_path, "w", encoding="utf-8") as txt_file:
            txt_file.write(ocr_result)

        print(f"Text gespeichert in: {txt_path}")


Text gespeichert in: data/txt/1851_Kölbing_Friedrich/28.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/29.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/30.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/31.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/32.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/33.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/34.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/35.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/36.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/37.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/38.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/39.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/40.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/41.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/42.txt
Text gespeichert in: data/txt/1851_Kölbing_Friedrich/43.txt
Text gespeichert in: data/txt/1851_Kölbi

## NER-Erkennung und Erstellung von den XML-Dateien

### Grundstruktur von den XML-Dateien 

In [61]:
import re
import os
import xml.etree.ElementTree as ET
from xml.dom import minidom

In [62]:
# Zugang zu den TXT-Dateien wird definiert
base_folder = "data/txt"

In [63]:
#Textbereinigung
def clean_text(text):
    if not text:
        return ''
    text = text.replace('\n', ' ').replace('\t', ' ')
    text = re.sub(r'(?<=[A-Za-z])<', 'ch', text)
    text = re.sub(r'(?<=[A-Za-z])>', 'ck', text)
    text = (text.replace("ic)", "ich")
                .replace("auc)", "auch")
                .replace("nac)", "nach")               
                .replace("aud)", "auch")
                .replace("Jh", "Ich")
                .replace("IJ)", "Ich")
                .replace("nad)", "nach")
                .replace("» ", "")
                .replace("- ", "")
                .replace(" —", " ")
                .replace("| ", " ")
                .replace("id)", "ich")
                .replace('!', '')
                .replace('/', '')
                .replace('-', '')
                .replace("'", '')
                .replace(',', '')
                .replace("— ", " ")
                .replace("„", "")
                .replace("“", ""))
    text = ' '.join(text.split())
    return text.strip()

In [64]:
# Die Dateien werden iteriert
for folder_name in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder_name)

    if not os.path.isdir(folder_path):
        continue

    txt_files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith('.txt')])

    if not txt_files:
        print(f"Keine .txt-Dateien in Ordner: {folder_name}")
        continue
    
    # === Erstelle die Root-Element der XML-Struktur ===
    root = ET.Element("TEI")
    root.set("version", "4.8.1")
    root.set("xmlns", "http://www.tei-c.org/ns/1.0")
    
        # teiHeader-Struktur
    headerTEI_el = ET.SubElement(root, "teiHeader")
    fileDesc_el = ET.SubElement(headerTEI_el, "fileDesc")
    titleStmt_el = ET.SubElement(fileDesc_el, "titleStmt")
    
    titleStmt_el.append(ET.Comment("Titel muss definiert werden. Nach der Korrektur wird der Kommentar gelöscht"))
    ET.SubElement(titleStmt_el, "title")

    titleStmt_el.append(ET.Comment("Autor muss definiert werden. Nach der Korrektur wird der Kommentar gelöscht"))
    ET.SubElement(titleStmt_el, "author")

    # respStmt
    respStmt_el = ET.SubElement(titleStmt_el, "respStmt")
    resp_el = ET.SubElement(respStmt_el, "resp")
    resp_el.text = "XML-Modelling compiled by"
    name_el = ET.SubElement(respStmt_el, "name")
    name_el.text = "Svetlana Yakutina"

    # publicationStmt
    publicationStmt_el = ET.SubElement(fileDesc_el, "publicationStmt")
    publisher_el = ET.SubElement(publicationStmt_el, "publisher")
    orgName_el = ET.SubElement(publisher_el, "orgName")
    orgName_el.text = "Verlag der Unitäts-Buchhandlung"

    pubPlace_el = ET.SubElement(publicationStmt_el, "pubPlace")
    pubPlace_el.text = "Gnadau (Germany)"

    availability_el = ET.SubElement(publicationStmt_el, "availability")
    p_header_el = ET.SubElement(availability_el, "p")
    orgName_el_ = ET.SubElement(p_header_el, "orgName")
    orgName_el_.text = "Memorial University of Newfoundland"

    publicationStmt_el.append(ET.Comment("Date muss definiert werden. Nach der Korrektur wird der Kommentar gelöscht"))
    ET.SubElement(publicationStmt_el, "date")

    publicationStmt_el.append(ET.Comment("Ref muss definiert werden. Nach der Korrektur wird der Kommentar gelöscht"))
    ET.SubElement(publicationStmt_el, "ref")

    # sourceDesc Struktur
    sourceDesc_el = ET.SubElement(fileDesc_el, "sourceDesc")
    bibl_el = ET.SubElement(sourceDesc_el, "bibl", type="j")
    bibl_el.text = "Nachrichten aus der Brüder-Gemeine"

    # biblFull Struktur
    biblFull_el = ET.SubElement(sourceDesc_el, "biblFull")
    titleStmt_el_ = ET.SubElement(biblFull_el, "titleStmt")
    ET.SubElement(titleStmt_el_, "title").text = "Nachrichten aus der Brüder-Gemeine"
    ET.SubElement(titleStmt_el_, "author").text = "Unbekannt"

    editionStmt_el = ET.SubElement(biblFull_el, "editionStmt")
    ET.SubElement(editionStmt_el, "edition").text = "Digitale Ausgabe der Zeitschrift"

    publicationStmt_el_ = ET.SubElement(biblFull_el, "publicationStmt")
    ET.SubElement(publicationStmt_el_, "publisher").text = "Verlag der Unitäts-Buchhandlung"

    seriesStmt_el = ET.SubElement(biblFull_el, "seriesStmt")
    title_el__ = ET.SubElement(seriesStmt_el, "title", level="j", type="main")
    title_el__.text = "Nachrichten aus der Brüder-Gemeine"
    ET.SubElement(seriesStmt_el, "biblScope", unit="volume").text = "1856-1894"
    seriesStmt_el.append(ET.Comment("Erscheinungsjahr muss definiert werden"))
    seriesStmt_el.append(ET.Comment("Seiten müssen definiert werden"))
    ET.SubElement(seriesStmt_el, "biblScope", unit="issue")
    ET.SubElement(seriesStmt_el, "biblScope", unit="page")

    notesStmt_el = ET.SubElement(biblFull_el, "notesStmt")
    note_el = ET.SubElement(notesStmt_el, "note", type="fileFormat")
    note_el.text = "application/pdf"

    # msDesc Struktur
    msDesc_el = ET.SubElement(sourceDesc_el, "msDesc")
    msIdentifier_el = ET.SubElement(msDesc_el, "msIdentifier")
    ET.SubElement(msIdentifier_el, "repository").text = "Memorial University of Newfoundland"
    idno_el = ET.SubElement(msIdentifier_el, "idno")
    idno_el_ = ET.SubElement(idno_el, "idno", type="URLCatalogue")
    idno_el_.text = "https://dai.mun.ca/digital/nachrichten/"

    # === Textstruktur ===
    text_el = ET.SubElement(root, "text")
    body_el = ET.SubElement(text_el, "body")
    div_el = ET.SubElement(body_el, "div")

    # Durchlaufe alle TXT-Dateien
    for txt_file in txt_files:
        txt_file_path = os.path.join(folder_path, txt_file)
        with open(txt_file_path, 'r', encoding='utf-8') as f:
            raw_text = f.read()
        clean = clean_text(raw_text)

        file_base = os.path.splitext(txt_file)[0]
        pb_el = ET.SubElement(div_el, "pb", n=file_base)

        paragraphs = clean.split('\n\n')
        for para in paragraphs:
            para = para.strip()
            if para:
                p_el = ET.SubElement(div_el, "p")
                p_el.text = para
                
    # XML-Datei erzeugen
    xml_str = ET.tostring(root, encoding='utf-8')
    pretty_xml = minidom.parseString(xml_str).toprettyxml(indent="  ")

    os.makedirs("data/xml", exist_ok=True)
    output_filename = f"{folder_name}.xml"
    output_path = os.path.join("data/xml", output_filename)

    with open(output_path, 'w', encoding='utf-8') as output_file:
        output_file.write(pretty_xml)

    print(f"XML-Datei erstellt: {output_path}")


XML-Datei erstellt: data/xml/1851_Kölbing_Friedrich.xml
XML-Datei erstellt: data/xml/1860_Suhl_D_W.xml
XML-Datei erstellt: data/xml/1847_Schmitt_J_H.xml
XML-Datei erstellt: data/xml/1856_Lemmerz_J.xml
XML-Datei erstellt: data/xml/1855_Hoffmann_M_E.xml
XML-Datei erstellt: data/xml/1893_Bonatz_J_A.xml
XML-Datei erstellt: data/xml/1875_Breutel_J_C.xml
XML-Datei erstellt: data/xml/1849_Hoffman_Johannes_Friedrich.xml
XML-Datei erstellt: data/xml/1820_Schulz_Johann_Gottlieb.xml
XML-Datei erstellt: data/xml/1884_Stuhl_S_E.xml
XML-Datei erstellt: data/xml/1862_Lemmerz_A.xml
XML-Datei erstellt: data/xml/1861_Kölbing_C_R.xml
XML-Datei erstellt: data/xml/1862_Beck_J_C.xml
XML-Datei erstellt: data/xml/1823_Bonatz_Johanna.xml
XML-Datei erstellt: data/xml/1889_Wedemann_J_F.xml
XML-Datei erstellt: data/xml/1836_Küster_Johann_Adolph.xml
XML-Datei erstellt: data/xml/1830_Bonatz_J_G.xml
XML-Datei erstellt: data/xml/1859_Fritsch_Johannes.xml
XML-Datei erstellt: data/xml/1866_Nauhaus_Carl.xml
XML-Datei er

### NER-Erkennung

In [65]:
import spacy
import os
import re
from lxml import etree as ET

In [66]:
# spaCy-Modell importieren
try:
    model_path = "spacy_train_v1/output/model-best"  # Pfad zum gespeicherten Modell
    ner_model = spacy.load(model_path)
    print(f"Modell erfolgreich geladen von {model_path}")
except Exception as e:
    print(f"Fehler beim Laden des Modells: {e}")

Modell erfolgreich geladen von spacy_train_v1/output/model-best


In [67]:
# Daten importieren
input_dir = 'data/xml'

In [68]:
def clean_text(text):
    # Entferne Punkte direkt nach Zahlen (z.B. "1." → "1")
    text = re.sub(r'(?<=\d)\.(?=\s|$)', '', text)
    return text.strip()

def append_text(elem, text):
    if elem.text:
        elem.text += text
    else:
        elem.text = text

def mark_entities_et(p_elem, ner_model):
    if p_elem.text is None and len(p_elem) == 0:
        return

    # Ursprünglichen Text extrahieren
    full_text = "".join(p_elem.itertext())
    cleaned_text = clean_text(full_text)

    # Element leeren
    p_elem.clear()

    # Sätze splitten
    sentences = [s.strip() for s in cleaned_text.split('. ') if s.strip()]

    for sentence in sentences:
        doc = ner_model(sentence)
        ents = [ent for ent in doc.ents if ent.label_ in {"PER", "LOC", "DATE"}]

        # Kein NER-Treffer: Satz einfach als Text anhängen
        if not ents:
            append_text(p_elem, sentence + ". ")
            continue

        # Satz mit <s>-Tag
        s_elem = ET.Element("s")
        last_idx = 0

        for ent in ents:
            # Text vor Entität
            if ent.start_char > last_idx:
                append_text(s_elem, sentence[last_idx:ent.start_char])

            # Entität mit passendem Tag
            tag = {"PER": "persName", "LOC": "placeName", "DATE": "date"}[ent.label_]
            ent_elem = ET.Element(tag)
            ent_elem.text = ent.text
            s_elem.append(ent_elem)

            last_idx = ent.end_char

        # Text nach letzter Entität
        if last_idx < len(sentence):
            append_text(s_elem, sentence[last_idx:])

        # Punkt am Satzende
        append_text(s_elem, ". ")

        # <s>-Element an Absatz anhängen
        p_elem.append(s_elem)


In [69]:
def append_text(parent, text):
    """Hilfsfunktion: hängt Text an letztes Element oder Parent"""
    if len(parent) > 0:
        # Hänge an tail des letzten Elements an
        if parent[-1].tail:
            parent[-1].tail += text
        else:
            parent[-1].tail = text
    else:
        # Hänge direkt an text des Elternelements
        if parent.text:
            parent.text += text
        else:
            parent.text = text


In [70]:
# Namespace-Map für TEI
ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

input_dir = 'data/xml'

# Durch alle XML-Dateien im Eingabeverzeichnis iterieren
for filename in os.listdir(input_dir):
    if filename.endswith(".xml"):
        file_path = os.path.join(input_dir, filename)

        try:
            # Lade das XML mit lxml-PARSER
            parser = ET.XMLParser(remove_blank_text=True)
            tree = ET.parse(file_path, parser)
            root = tree.getroot()

            # Alle <p>-Elemente finden (inkl. Namespace)
            p_elems = root.xpath('.//tei:text//tei:p', namespaces=ns)
            for p in p_elems:
                mark_entities_et(p, ner_model)  # ← Deine NER-Funktion

            # XML-Datei **überschreiben** mit schön formatiertem XML
            tree.write(file_path, encoding='utf-8', pretty_print=True, xml_declaration=True)
            print(f"XML-Datei wurde aktualisiert: {file_path}")

        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {filename}: {e}")


XML-Datei wurde aktualisiert: data/xml/1893_Bonatz_J_A.xml
XML-Datei wurde aktualisiert: data/xml/1856_Lemmerz_J.xml
XML-Datei wurde aktualisiert: data/xml/1820_Schulz_Johann_Gottlieb.xml
XML-Datei wurde aktualisiert: data/xml/1836_Küster_Johann_Adolph.xml
XML-Datei wurde aktualisiert: data/xml/1847_Schmitt_J_H.xml
XML-Datei wurde aktualisiert: data/xml/1859_Fritsch_Johannes.xml
XML-Datei wurde aktualisiert: data/xml/1851_Kölbing_Friedrich.xml
XML-Datei wurde aktualisiert: data/xml/1866_Nauhaus_Carl.xml
XML-Datei wurde aktualisiert: data/xml/1862_Schulz_J_B.xml
XML-Datei wurde aktualisiert: data/xml/1855_Hoffmann_M_E.xml
XML-Datei wurde aktualisiert: data/xml/1862_Lemmerz_A.xml
XML-Datei wurde aktualisiert: data/xml/1889_Wedemann_J_F.xml
XML-Datei wurde aktualisiert: data/xml/1830_Bonatz_J_G.xml
XML-Datei wurde aktualisiert: data/xml/1861_Kölbing_C_R.xml
XML-Datei wurde aktualisiert: data/xml/1884_Stuhl_S_E.xml
XML-Datei wurde aktualisiert: data/xml/1875_Breutel_J_C.xml
XML-Datei wurde

## Relation-Erkennung

### Daten vorbereiten 

#### Extraktion der erwähnten Personen-, Organisations- und Ortsnamen 

In [178]:
import os
import json
import re
from lxml import etree as ET

In [179]:
# Namespace muss außerhalb des Imports definiert werden
ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

In [180]:
input_dir = 'data/xml_kor'
output_file = 'data/json/erwähnte_NER.json'

In [181]:
# Funktion zum Säubern der Texte (Zeilenumbrüche, Leerzeichen etc.)
def clean_text(text):
    if not text:
        return ''
    text = re.sub(r'\s+', ' ', text)  # mehrere Whitespaces durch ein Leerzeichen ersetzen
    return text.strip()

In [182]:
all_results = []

In [183]:
for filename in os.listdir(input_dir):
    if filename.endswith(".xml"):
        file_path = os.path.join(input_dir, filename)

        try:
            parser = ET.XMLParser(remove_blank_text=True)
            tree = ET.parse(file_path, parser)
            root = tree.getroot()
            
            # Autor extrahieren (nur erster Autor oder leere Zeichenkette)
            author_element = root.xpath('.//tei:teiHeader//tei:author', namespaces=ns)
            autor = ''
            if author_element and author_element[0].text:
                autor = clean_text(author_element[0].text)

            p_elems = root.xpath('.//tei:text//tei:p', namespaces=ns)

            personen = set()
            orte = set()
            organisationen = set()

            for p in p_elems:
                pers_names = p.xpath('.//tei:persName[not(ancestor::tei:s)]', namespaces=ns)
                org_names = p.xpath('.//tei:orgName[not(ancestor::tei:s)]', namespaces=ns)
                place_names = p.xpath('.//tei:placeName[not(ancestor::tei:s)]', namespaces=ns)

                for pn in pers_names:
                    if pn.text:
                        personen.add(clean_text(pn.text))

                for on in org_names:
                    if on.text:
                        organisationen.add(clean_text(on.text))

                for pln in place_names:
                    if pln.text:
                        orte.add(clean_text(pln.text))

            result = {
                "datei": filename,
                "autor": autor,
                "personen": sorted(list(personen)),
                "orte": sorted(list(orte)),
                "organisationen": sorted(list(organisationen))
            }

            all_results.append(result)

        except ET.XMLSyntaxError as e:
            print(f"Fehler beim Parsen von {filename}: {e}")
        except Exception as e:
            print(f"Anderer Fehler bei {filename}: {e}")

# Sicherstellen, dass das Ausgabe-Verzeichnis existiert
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Schreibe alles in eine JSON-Datei
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)

print(f"Ergebnisse wurden in {output_file} gespeichert.")

Ergebnisse wurden in data/json/erwähnte_NER.json gespeichert.


#### Sätzenextraktion 

In [18]:
import os
import json
import re
from lxml import etree as ET

In [19]:
# Pfade
input_dir = 'data/xml_kor'
output_file = 'data/json/sätze.json'

In [20]:
# XML-Namespaces
ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

In [21]:
# Funktion zur Textbereinigung
def clean_text(text):
    if not text:
        return ''
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [22]:
all_results = []

# Alle XML-Dateien im Verzeichnis durchgehen
for filename in os.listdir(input_dir):
    if filename.endswith(".xml"):
        file_path = os.path.join(input_dir, filename)

        try:
            parser = ET.XMLParser(remove_blank_text=True)
            tree = ET.parse(file_path, parser)
            root = tree.getroot()

            # Metadaten: Autor und Referenz
            author_element = root.xpath('.//tei:teiHeader//tei:author', namespaces=ns)
            autor = clean_text(author_element[0].text) if author_element and author_element[0].text else ''

            ref_element = root.xpath('.//tei:teiHeader//tei:ref', namespaces=ns)
            ref = clean_text(ref_element[0].text) if ref_element and ref_element[0].text else ''

            # HEADS verarbeiten
            head_elements = root.xpath('.//tei:text//tei:head', namespaces=ns)
            for head in head_elements:
                head_text = clean_text(''.join(head.itertext()))

                head_personen = set()
                head_organisationen = set()
                head_orte = set()
                head_date = set()

                for pn in head.xpath('.//tei:persName', namespaces=ns):
                    if pn.text:
                        head_personen.add(clean_text(pn.text))
                for on in head.xpath('.//tei:orgName', namespaces=ns):
                    if on.text:
                        head_organisationen.add(clean_text(on.text))
                for pln in head.xpath('.//tei:placeName', namespaces=ns):
                    if pln.text:
                        head_orte.add(clean_text(pln.text))
                for dt in head.xpath('.//tei:date', namespaces=ns):
                    if dt.text:
                        head_date.add(clean_text(dt.text))

                result = {
                    "text": head_text,
                    "personen": sorted(head_personen),
                    "orte": sorted(head_orte),
                    "organisationen": sorted(head_organisationen),
                    "date": sorted(head_date),
                    "datei": filename,
                    "autor": autor,
                    "ref": ref
                }
                all_results.append(result)

            # SÄTZE (s-Tags) verarbeiten
            s_elements = root.xpath('.//tei:text//tei:s', namespaces=ns)
            for s in s_elements:
                s_text = clean_text(''.join(s.itertext()))

                s_personen = set()
                s_organisationen = set()
                s_orte = set()
                s_date = set()

                for pn in s.xpath('.//tei:persName', namespaces=ns):
                    if pn.text:
                        s_personen.add(clean_text(pn.text))
                for on in s.xpath('.//tei:orgName', namespaces=ns):
                    if on.text:
                        s_organisationen.add(clean_text(on.text))
                for pln in s.xpath('.//tei:placeName', namespaces=ns):
                    if pln.text:
                        s_orte.add(clean_text(pln.text))
                for dt in s.xpath('.//tei:date', namespaces=ns):
                    if dt.text:
                        s_date.add(clean_text(dt.text))

                result = {
                    "text": s_text,
                    "personen": sorted(s_personen),
                    "orte": sorted(s_orte),
                    "organisationen": sorted(s_organisationen),
                    "date": sorted(s_date),
                    "datei": filename,
                    "autor": autor,
                    "ref": ref
                }
                all_results.append(result)

        except ET.XMLSyntaxError as e:
            print(f"Fehler beim Parsen von {filename}: {e}")
        except Exception as e:
            print(f"Anderer Fehler bei {filename}: {e}")

In [23]:
# Sicherstellen, dass das Ausgabe-Verzeichnis existiert
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Ergebnisse als JSON speichern
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)

print(f"Ergebnisse wurden in {output_file} gespeichert.")


Ergebnisse wurden in data/json/sätze.json gespeichert.


### Relation erkennen

In [58]:
import json
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser

In [59]:
# Neue Entity- und Relationstypen
entity_types = ['person', 'location', 'organization', 'date']

relation_types = [
    # Biografische Basisdaten
    "geboren_in", "geboren_am",
    "gestorben_in", "gestorben_am",
    "begraben_in",

    # Wohn- und Wirkorte
    "wohnhaft_in", "lebte_in", "wirkte_in", "aufenthalt_in",

    # Bildung und Beruf
    "ausgebildet_in", "studierte_an",
    "lehre_als", "lehre_bei",
    "tätig_als", "tätig_bei",
    "war", "beschäftigt_bei", "mitglied_von",
    "war_eingeschult", "unterrichtet_in",

    # Familie
    "verheiratet_mit", "kind_von", "eltern_von", "geschwister_von",

    # Religion & Kirche
    "getauft_am", "beigetreten_am", "konfirmiert_am", "hat_heiliges_abendmahl",

    # Interessen & Themen
    "interessiert_sich_für", "beschäftigt_sich_mit", "forscht_zu", "schreibt_über",

    # Reisen & Aufenthalte
    "gereist_nach", "besucht", "angekommen_in", "abgereist_aus",
    "aufgehalten_in", "zurückgekehrt_nach", "verließ", "war_in", "war_am", "gefahren_nach"]

In [60]:
class ExtractedInfo(BaseModel):
    subjekt: str = Field(description="extrahierte Subjekt-Entität, z.B. Daniel Wilhelm Suhl, ich, er, wir, meine l. Frau, seliger Bruder")
    subjekt_type: str = Field(description="Typ der Subjekt-Entität, z.B. person")
    prädikat: str = Field(description="Beziehung zwischen Subjekt und Objekt")
    objekt: str = Field(description="extrahierte Objekt-Entität, z.B. Gnadenthal, Aeltesten-Konferenz")
    objekt_type: str = Field(description="Typ der Objekt-Entität, z.B. location, organisation")
    zeit: str = Field(default=None, description="Datumsangabe zum Ereignis, z.B. 30 April 1858 oder leer, falls nicht genannt")

In [61]:
# Parser mit neuem Modell
parser = PydanticOutputParser(pydantic_object=ExtractedInfo)


In [62]:
# Systemprompt
system_prompt = PromptTemplate(
    template="""
Du arbeitest mit biographischen, historischen und religiösen Texten der Herrnhuter Brüdergemeine.
Deine Aufgabe ist es, Relationen zwischen Personen, Orten, Organisationen und Daten zu extrahieren.

Gib ausschließlich eine JSON-Liste mit Objekten zurück. Jedes Objekt enthält:
- "subjekt", "subjekt_type", "prädikat", "objekt", "objekt_type", "zeit".

Die erlaubten Entitätstypen sind: {entity_types}
Die erlaubten Beziehungstypen sind: {relation_types}

KEINE Erklärungen oder zusätzlichen Texte!
""",
    input_variables=["entity_types", "relation_types"]
)
system_message_prompt = SystemMessagePromptTemplate(prompt=system_prompt)

In [63]:
# Humanprompt
human_prompt = PromptTemplate(
    template="""
Hier ist ein Satz aus einem historischen Lebenslauf:
"{text}"

Folgende Entitäten sind im Satz enthalten:
personen: {personen}
orte: {orte}
date: {date}
organisationen: {organisationen}

Nutze nur die angegebenen Entitäten. Finde sinnvolle Relationen zwischen diesen Entitäten, z. B. wann und wo jemand geboren oder gestorben ist, wo jemand lebte oder tätig war.

Wenn sinnvoll, gib zusätzlich im Feld "zeit" das relevante Datum an.

{format_instructions}
""",
    input_variables=["text", "personen", "orte", "date", "organisationen"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

In [64]:
# Chatmodell + Prompt-Kette
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
model = ChatOllama(model="llama3", temperature=0)
chain = LLMChain(llm=model, prompt=chat_prompt)


In [65]:
# Lade Eingabedaten
with open("data/json/sätze.json", "r", encoding="utf-8") as f:
    input_data = json.load(f)

In [ ]:
results = []

# Extraktion starten
for entry in input_data:
    result = chain.run(
        entity_types=entity_types,
        relation_types=relation_types,
        text=entry["text"],
        personen=", ".join(entry.get("personen", [])),
        orte=", ".join(entry.get("orte", [])),
        date=", ".join(entry.get("date", [])),
        organisationen=", ".join(entry.get("organisationen", []))
    )
    results.append({
        "input": entry,
        "extracted_relations": result
    })

# Speichere Ergebnisse
with open("data/json/graph.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Extraktion abgeschlossen. Ergebnisse gespeichert in 'graph.json'.")

Extraktion abgeschlossen. Ergebnisse gespeichert in 'extrahierte_relationen.json'.


In [67]:
import json

# Input-Datei laden
with open("data/json/graph.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

# Ergebnisliste initialisieren
output = []

for entry in data:
    input_data = entry.get("input", {})
    relations_str = entry.get("extracted_relations", "[]")
    
    try:
        relations = json.loads(relations_str)
    except json.JSONDecodeError:
        relations = []

    transformed = {
        "text": input_data.get("text", ""),
        "ref": input_data.get("ref", ""),
        "datei": input_data.get("datei", ""),
        "autor": input_data.get("autor", ""),
        "graph": relations  # genau hier: Liste von Relationen
    }

    output.append(transformed)

# Ausgabe-Datei schreiben
with open('data/json/graphen.json', 'w', encoding='utf-8') as f:
    json.dump(output, f, indent=2, ensure_ascii=False)


## Daten anreichnern 

In [1]:
import json
import pandas as pd
import requests

In [41]:
with open("data/json/graphen.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [42]:
# DataFrame strukturieren
records = []
for entry in data:
    for g in entry.get("graph", []):
        records.append({
            "text": entry.get("text", ""),
            "ref": entry.get("ref", ""),
            "nbg": "",
            "datei": entry.get("datei", ""),
            "autor": entry.get("autor", ""),
            "subjekt": g.get("subjekt", ""),
            "subjekt_type": g.get("subjekt_type", ""),
            "q_subjekt": "",
            "prädikat": g.get("prädikat", ""),
            "p_wert": "",
            "objekt": g.get("objekt", ""),
            "objekt_type": g.get("objekt_type", ""),
            "q_objekt":"",
            "zeit": g.get("zeit", "")
        })


In [43]:
df = pd.DataFrame(records)
df.to_excel("data/json/graphen_1.xlsx", index=False)

In [ ]:
#df = pd.read_excel("data/json/graphen_.xlsx")

In [ ]:
#Q-ID (Entitäten)
def get_wikidata_qid(label):
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": label,
        "language": "de",
        "format": "json",
        "type": "item",
        "limit": 1
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    if data.get("search"):
        return data["search"][0]["id"]
    return ""

#P-ID (Properties)
def get_property_id(label):
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": label,
        "language": "de",
        "format": "json",
        "type": "property",
        "limit": 1
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    if data.get("search"):
        return data["search"][0]["id"]
    return ""

In [ ]:
# Q-ID für Subjekte
unique_subjekte = df["subjekt"].unique()
qid_map_subjekt = {}
for subj in unique_subjekte:
    qid = get_wikidata_qid(subj)
    qid_map_subjekt[subj] = qid

df["q_subjekt"] = df["subjekt"].map(qid_map_subjekt)

# Q-ID für Objekte (achte auf den richtigen Spaltennamen: q_objekt)
unique_objekte = df["objekt"].unique()
qid_map_objekt = {}
for obj in unique_objekte:
    qid = get_wikidata_qid(obj)
    qid_map_objekt[obj] = qid

df["q_objekt"] = df["objekt"].map(qid_map_objekt)

# P-ID für Prädikate (nutze get_property_id!)
unique_prädikate = df["prädikat"].unique()
pid_map = {}
for prd in unique_prädikate:
    pid = get_property_id(prd)
    pid_map[prd] = pid

df["p_wert"] = df["prädikat"].map(pid_map)

In [ ]:
#df.to_excel("data/json/graphen_kor.xlsx", index=False)

In [2]:
df_person = pd.read_excel("data/personen.xlsx")

In [3]:
df_weg = pd.read_excel("data/weg.xlsx")

In [22]:
def get_coordinates(qid):
    if not qid or not isinstance(qid, str) or not qid.startswith("Q"):
        return None, None

    url = "https://query.wikidata.org/sparql"
    query = f"""
    SELECT ?coord WHERE {{
      wd:{qid} wdt:P625 ?coord.
    }}
    """
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "GeoKoordinatenBot/1.0 (example@domain.com)"  # Eigene Adresse oder Projektname
    }

    try:
        response = requests.get(url, params={"query": query}, headers=headers, timeout=10)
        response.raise_for_status()

        data = response.json()
        bindings = data.get("results", {}).get("bindings", [])
        if bindings:
            coord_str = bindings[0]["coord"]["value"]
            lon, lat = coord_str.replace("Point(", "").replace(")", "").split()
            return float(lat), float(lon)
    except Exception as e:
        print(f"Fehler bei {qid}: {e}")
    return None, None


In [ ]:
# Optional: Nur für geografische Entitäten (objekt_type == "location")
df_weg = df[df["objekt_type"] == "location"].copy()

# Koordinaten abrufen
df_weg["lat"], df_weg["lon"] = zip(*df_weg["q_objekt"].map(get_coordinates))

# Ergebnis zurück in Haupt-DataFrame schreiben
df.loc[df_weg.index, "lat"] = df_weg["lat"]
df.loc[df_weg.index, "lon"] = df_weg["lon"]



In [4]:
import pandas as pd
import requests

# Excel-Datei laden
df_weg = pd.read_excel("data/weg.xlsx")

# Funktion zum Abrufen der Koordinaten aus Wikidata
def get_coordinates(qid):
    if not qid or not isinstance(qid, str) or not qid.startswith("Q"):
        return None, None

    url = "https://query.wikidata.org/sparql"
    query = f"""
    SELECT ?coord WHERE {{
      wd:{qid} wdt:P625 ?coord.
    }}
    """
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "GeoKoordinatenBot/1.0 (example@domain.com)"
    }

    try:
        response = requests.get(url, params={"query": query}, headers=headers, timeout=10)
        response.raise_for_status()

        data = response.json()
        bindings = data.get("results", {}).get("bindings", [])
        if bindings:
            coord_str = bindings[0]["coord"]["value"]
            lon, lat = coord_str.replace("Point(", "").replace(")", "").split()
            return float(lat), float(lon)
    except Exception as e:
        print(f"Fehler bei {qid}: {e}")
        return None, None

# Nur Zeilen mit objekt_type == "location"
df_locations = df_weg[df_weg["objekt_type"] == "location"].copy()

# Koordinaten abrufen
koordinaten = df_locations["q_objekt"].map(get_coordinates)
df_locations["lat"], df_locations["lon"] = zip(*koordinaten)

# Zurückschreiben in df_weg
df_weg.loc[df_locations.index, "lat"] = df_locations["lat"]
df_weg.loc[df_locations.index, "lon"] = df_locations["lon"]


In [13]:
df_weg.to_excel("data/weg_mit_koordinaten.xlsx", index=False)
df_weg.to_json("weg.json", orient="records", indent=2, force_ascii=False)

In [15]:
df_person = pd.read_excel("data/personen.xlsx")
df_person.to_json("personen.json", orient="records", indent=2, force_ascii=False)

## Dateikonvertierung

### Weg

In [2]:

import json

# Eingabedatei laden
with open("weg.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# Personenrouten aufbauen (als Liste)
personen_routes = {}

for eintrag in raw_data:
    person = eintrag.get("subjekt")
    id_person = eintrag.get("subjekt_ref")
    location = eintrag.get("objekt")
    location_type = eintrag.get("prädikat")
    id_location = eintrag.get("objekt_ref")
    lat = eintrag.get("lat")
    lng = eintrag.get("lon")
    date = eintrag.get("zeit")  # Optional, kein Umwandeln nötig

    if not (person and location and lat and lng):
        continue  # Unvollständige Einträge überspringen

    if person not in personen_routes:
        personen_routes[person] = {
            "person": person,
            "person_id": id_person,
            "color": "",  # Du kannst später im JS eine Farbe setzen
            "route": []
        }

    personen_routes[person]["route"].append({
        "location": location,
        "location_id": id_location,
        "location_type": location_type,
        "lat": lat,
        "lng": lng,
        "date": date  # ggf. None oder z. B. "Mai 1842"
    })

# Liste erzeugen und exportieren
result = list(personen_routes.values())

with open("leaflet_routes.json", "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

print("✅ 'leaflet_routes.json' wurde erstellt.")


✅ 'leaflet_routes.json' wurde erstellt.


## RDF / OWL


# Test


In [3]:
import pandas as pd
import json



df_person = pd.read_excel("data/person_1.xlsx")
df_person.to_json("personen_.json", orient="records", indent=2, force_ascii=False)